<a href="https://colab.research.google.com/github/icelaye/MCDAustral-TP-RA/blob/main/LLaMA_4_bit_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLaMA 4 bit inference

LLaMA 4-bit is a version of the LLaMA language model that has been optimized for improved performance and reduced memory requirements.

![outpaint](https://substackcdn.com/image/fetch/w_1456,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F11b15c1c-34b8-46d9-bda3-383c36b37e66_1278x340.png)

| Code Credits | Link |
| ----------- | ---- |
| 🎉 Repository | [![GitHub Repository](https://img.shields.io/github/stars/facebookresearch/llama?style=social)](https://github.com/facebookresearch/llama) |
| 🚀 Online inference | [![Hugging Face Spaces](https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Spaces-blue)](https://huggingface.co/spaces/trl-lib/stack-llama) |
| 4 bits quantization of LLaMa using GPTQ | [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-black?style=flat-square&logo=github)](https://github.com/qwopqwop200/GPTQ-for-LLaMa/) |
| Text-generation-webui | [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-black?style=flat-square&logo=github)](https://github.com/oobabooga/text-generation-webui/) |
| Colab gradio web UI | [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-black?style=flat-square&logo=github)](https://github.com/camenduru/text-generation-webui-colab/) |
| 🔥 Discover More Colab Notebooks | [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-black?style=flat-square&logo=github)](https://github.com/R3gm/Colab-resources/) |


# Install libraries

In [ ]:
!pip uninstall -q -y torch triton torchtext tensorflow tensorflow-metadata
!pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install git+https://github.com/huggingface/transformers ##
!pip install safetensors==0.3.0 \
    datasets==2.10.1 \
    sentencepiece \
    git+https://github.com/zphang/transformers@660dd6e2bbc9255aacd0e60084cf15df1b6ae00d \
    accelerate==0.17.1 \
    triton==2.0.0 \
    texttable \
    toml \
    numpy \
    protobuf==3.20.2

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.8/1.8 GB 173.8 MB/s eta 0:00:01

`RESTART RUNTIME`

# Setup

In [ ]:
!git clone https://github.com/qwopqwop200/GPTQ-for-LLaMa
%cd GPTQ-for-LLaMa
!git checkout 9bbfdeda2c80c20d8bc1babf4d4368df574afe30

# download model
!wget https://huggingface.co/decapoda-research/llama-7b-hf-int4/resolve/main/llama-7b-4bit.pt

# setup quant_cuda
!python ./setup_cuda.py install

Cloning into 'GPTQ-for-LLaMa'...
remote: Enumerating objects: 1858, done.
remote: Counting objects: 100% (618/618), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 1858 (delta 519), reused 489 (delta 445), pack-reused 1240
Receiving objects: 100% (1858/1858), 2.74 MiB | 12.60 MiB/s, done.
Resolving deltas: 100% (1055/1055), done.
/content/GPTQ-for-LLaMa
Note: switching to '9bbfdeda2c80c20d8bc1babf4d4368df574afe30'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 9bbfded Update README.md
-

# Inference
- This type of inference loads the model every time, which is not recommended.

In [ ]:
!CUDA_VISIBLE_DEVICES=0 \
    python llama_inference.py \
    decapoda-research/llama-7b-hf --wbits 4 --load llama-7b-4bit.pt \
    --text 'What is inflation and how does it impact the economy?'

Loading model ...
Done.
What is inflation and how does it impact the economy?
Inflation is the measure of the overall level of price increases in the economy. Inflation is the increase in the general level of prices in an economy over a given period of


In [ ]:
!CUDA_VISIBLE_DEVICES=0 \
    python llama_inference.py \
    decapoda-research/llama-7b-hf --wbits 4 --load llama-7b-4bit.pt \
    --text 'Translate: The only limit to our realization of tomorrow will be our doubts of today\n To spanish:'

Loading model ...
Done.
Translate: The only limit to our realization of tomorrow will be our doubts of today\n To spanish: El unico límite a nuestro realce del mañana será nuestra duda de ho


# Inference with the Pipeline

In [ ]:
%cd GPTQ-for-LLaMa

/content/GPTQ-for-LLaMa


In [ ]:
# Load model
import time
import torch
import torch.nn as nn
from gptq import *
from modelutils import *
from quant import *
from transformers import AutoTokenizer
from llama_inference import get_llama, load_quant

device = torch.device('cuda:0')

llama4bits = True
model_llama = 'decapoda-research/llama-7b-hf'
chekpoint_model = 'llama-7b-4bit.pt'

if llama4bits:
    model = load_quant(model_llama, chekpoint_model, 4)
else:
    model = get_llama(model_llama)
    model.eval()

model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_llama)

Loading model ...
Done.


In [ ]:
model

LLaMAForCausalLM(
  (model): LLaMAModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LLaMADecoderLayer(
        (self_attn): LLaMAAttention(
          (q_proj): QuantLinear()
          (k_proj): QuantLinear()
          (v_proj): QuantLinear()
          (o_proj): QuantLinear()
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): LLaMAMLP(
          (gate_proj): QuantLinear()
          (down_proj): QuantLinear()
          (up_proj): QuantLinear()
          (act_fn): SiLUActivation()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)

In [ ]:
prompt = 'Hello world'

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
input_ids

tensor([[    1, 15043,  3186]], device='cuda:0')

In [ ]:
# Inference
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=500,
        top_p=0.95,
        temperature=0.8,
    )

# Decode the tokenizer
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

Hello world! I’m a newbie to WordPress. I like the look of your blog. Is it a theme or a plugin?
Thanks for the compliment. It’s a theme. It’s called the Raven.
I would like to know how you are getting your blog to rank in google?
Thanks for the compliment. It’s a theme. It’s called the Raven. I am not sure about the google question. I don’t do that. I just write.
Previous PostPrevious What is a Raven?
